In [3]:
# Librerias generales
import pandas as pd
import numpy as np
import json

# Time
import datetime as dt
from datetime import datetime, timedelta

# Visualización
import seaborn as sns
import matplotlib.pyplot as plt
from dython.model_utils import roc_graph
from dython.nominal import associations
%matplotlib inline

# OneHotEncoding
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest, chi2 # for chi-squared feature selection
import prince

from sklearn import metrics

import os
import json
import joblib

from pandas.io.json import json_normalize
from datetime import datetime 
from tqdm import tqdm
from sklearn.preprocessing import PowerTransformer
import umap
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from scipy import stats

from sklearn.cluster import KMeans
from kmodes.kprototypes import KPrototypes


from lightgbm import LGBMClassifier
import shap
from sklearn.model_selection import cross_val_score


# Algoritmos
from sklearn.preprocessing import StandardScaler, normalize
from sklearn import preprocessing 
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as shc
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.cluster import MeanShift
from sklearn.cluster import estimate_bandwidth

# Pipeline
from sklearn.pipeline import make_pipeline

from jupyterthemes import jtplot
jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)

# Configuración de pandas 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
# new_data = Processing_data(data)
# new_data_cat = Processing_data(data_cath)
# Merge manual 

In [3]:
# 1. Leo data numerica
# 2. Leo data categorica
# 3. new_data = Processing_data(data, cath_data)
# 4. Ejecuta - merge cuando yo le diga

In [4]:
# Resultado

In [29]:
class Processing_data():
    """**Processing data**
    
    | When it comes to developing a Sempli cluster analyse we can use this class to do the work. To effectively develop a study you should know that this project comes with two steps:
    * Cleaning and executing the model
    * Testing the model
    
    | That being said, we are going to explain the steps to succesfully run our program:
    
    **Cleaning and executing the model**
    
    | The first step that you should do is read the csv or excel file providing the information (You can do that using the pandas library, and the methods read_csv or read_excel). In our case we have two provided datasets with finantial and cathegorical 
    information.
    
    | After you read those two files, and store the information in two different variables (data and cat_data) it's time to instance a variable with our class.
    
    **Example:**
    
        var_name = Procesing_data(data, cat_data)
    
    | On the example provided we pass the two datasets (data & cat_data). The steps to process the data are:
    
    | Processing data steps:
    
        1- After reading the file (excel or csv) fill n/a rows of the file on json format rows using -fill_na_json method-
        
        2- Clean the dataframe to have the first draft of you desired output using -clean_data method-
        
        3- Delete unnecessary columns on numerical data using -delete_unnecessary_columns method-
        
        4- Organize the dataframe to have you desired dataframe to study using -organizing_data-
        
        5- Check the cathegorical dataset to use only the important information on it using -checking_cat_data method-
        
        6- We do a first merge to only analyze clients withing the two dataframes using -merge_two_tables method-
        
        7- After checking the clients present in the datasets we delete the cathegorical information that would be added later using -deleting_cat_info method-
        
        8- At this point we use a method called -customer_level- to analyze the behaviour of clients on their six latest payment
        
        9- When we have the finantial information clean and without any error we merge the two tables againg using the same method
        of step 6
    
    | All of these steps are present on the data_cleaning method. It's imperative to clean the data using these steps provided below to avoid any confusion with the clients information.
    
    | After doing all the data cleaning you can execute the model using the execute_model method. By executing this method you don't have to worry about cleaning the data because it takes care of it but you need to be aware of telling the class to clean the datasets by passing True to the **cleaning** variable of the constructor.
    
    | After you executing the model you can save it by using the save_model method and you will create a **testeo.joblib** file which you will use on the next step that is testing the model.
    
    **Example:**
        
        >>> var_name.guardar_model()
    
    | Here it's an example on how to run the class: 
        **Cleaning and executing the model**
        
        >>> path_file = "loan_schedules_2.xlsx"
        >>> cols = ['client_id',
                'loan_schedule_id',
                'loan_id',
                'index',
                'expected_date',
                'state',
                'arrears_days',
                'expected_detail',
                'paid_detail']
        >>> data = pd.read_excel(path_file, sheet_name = 0, header = 0, usecols = cols).replace('NaT', np.nan)

        >>> path_file = "data_sempli_holberton.xlsx"
        >>> cat_cols = ['ID Cliente', 'Monto Acumulado', 'Uso de los recursos', 'Plazo',
                    'Sector', 'Ingresos', 'Ubicación', 'Estrato Mínimo',
                    'Procesos judiciales', 'Alertas', 'Score Bureau Empresa', 'Huellas de Consulta', 'Tiempo en el negocio',
                    'Website empresa', 'Instagram empresa', 'LinkedIn empresa',
                    'LinkedIn empresarios', 'Edad empresarios', 'Activador', 'Número de accionistas',
                    'Impacto', 'Acceso previso a la banca', '# Empleados',
                    'Mujeres empresarias', 'Mujeres en cargos directivos']
        >>> cat_data = pd.read_excel(path_file, sheet_name = 0, header = 2, usecols = cat_cols).replace('SIN INFO', np.nan)
        >>> cat_data = cat_data.dropna()
        
        >>> new_data = Processing_data(data, cat_data)
        
        >>> new_data.guardar_model()
    """

    def __init__(self, data, cat_data):
        """Constructor of attributes
        
        | data = finaltial/numerical information of clients.
        | cat_data = cathegorical information about clients. Released by the risk department.
        | kproto = In this variable we have the trained model
        | clusters = Centroids of the trained algorithm
        | cleaning = Holds a boolean (True/False). True = clean the dataset and save the model, False= Test the model
        """
        self.data = data
        self.kproto = []
        self.clusters = []
        self.cat_data = cat_data
        #self.cleaning = cleaning

    def fill_na_json(self):
        """Filling missing values in json format columns"""
        col_names = ['expected_detail', 'paid_detail']
        for col_name in col_names:
            self.data[col_name] = self.data[col_name].apply(
                lambda x: "{}" if pd.isna(x) else x)
        return self.data

    @staticmethod
    def normalize_data(df, col_name):
        data = df[col_name].apply(json.loads)
        return pd.json_normalize(data)

    def clean_data(self):
        """
        Cleaning process of data:
        1- normalize
        2- deleting useless state (GRACE, PENDING, AND PARTIALLY PAID) on data
        """
        expected = Processing_data.normalize_data(self.data, 'expected_detail')
        paid = Processing_data.normalize_data(self.data, 'paid_detail')
        self.data = self.data.join(expected).join(paid, rsuffix='_paid')
        self.data = self.data[self.data.state != "GRACE"]
        self.data = self.data[self.data.state != "PENDING"]
        self.data = self.data[self.data.state != "PARTIALLY_PAID"]
        return self.data

    def delete_unnecessary_columns(self):
        """Deletes unnecesary columns produced by the json format"""
        data_droped_columns = ['penalty', 'interest', 'insurance', 'principal', 'taxRetentionValue',
                               'taxRetentionPercentage', 'legalCharge', 'preLegalCharge', 'feesPaid',
                               'fngTotal', 'fngValue',
                               'fngPercent', 'fngVatValue', 'fngVatPercent', 'monthlyDeferredInterest',
                               'penalty_paid', 'interest_paid', 'insurance_paid',
                               'principal_paid', 'taxRetentionValue_paid',
                               'taxRetentionPercentage_paid', 'legalCharge_paid',
                               'preLegalCharge_paid', 'feesPaid_paid', 'fngTotal_paid',
                               'fngValue_paid', 'fngPercent_paid', 'fngVatValue_paid',
                               'fngVatPercent_paid', 'monthlyDeferredInterest_paid', 'expected_detail', 'paid_detail']
        self.data = self.data.drop(columns=data_droped_columns)
        return self.data

    def organizing_data(self):
        """
        Organize data after having the first draft of your dataframe and fills total na rows with 0
        and groups clients by their six newest loan_id
        This is step three
        """
        self.data = self.data.sort_values(
            ['client_id', 'loan_id', 'expected_date'])
        self.data['total'] = self.data['total'].fillna(0)
        self.data['total_paid'] = self.data['total_paid'].fillna(0)
        self.data = self.data.groupby('loan_id').filter(
            lambda x: x['loan_id'].value_counts() > 6).groupby('loan_id').tail(6)
        return self.data

    def checking_cat_data(self):
        """Checks for the clients that are present in the two tables"""
        self.cat_data = self.cat_data.sort_values(
            by='ID Cliente', ascending=True)

        rep_columns = ['Procesos judiciales', 'Alertas', 'Website empresa',
                       'Instagram empresa', 'LinkedIn empresa',
                       'LinkedIn empresarios', 'Impacto', 'Acceso previso a la banca',
                       'Mujeres empresarias']

        # Replace values for Si/No
        self.cat_data[rep_columns] = self.cat_data[rep_columns].replace({'No procesos judiciales': 'No',
                                                                         'Sí procesos judiciales': 'Si',
                                                                         'No Alertas': 'No',
                                                                         'Sí Alertas': 'Si',
                                                                         'No website': 'No',
                                                                         'Si website': 'Si',
                                                                         'No Ig': 'No',
                                                                         'Si Ig': 'Si',
                                                                         'No LinkedIn': 'No',
                                                                         'Si LinkedIn': 'Si',
                                                                         'No LinkedIn empresarios': 'No',
                                                                         'Si LinkedIn empresarios': 'Si',
                                                                         'Si Impacto': 'Si',
                                                                         'No Impacto': 'No',
                                                                         'Si acceso a la banca': 'Si',
                                                                         'No acceso a la banca': 'No',
                                                                         'No mujeres empresarias': 'No',
                                                                         'Si mujeres empresarias': 'Si'})
        return self.cat_data

    def deleting_cat_info(self):
        """Deletes cat_data to analize clients on mean"""
        cat_cols = ['ID Cliente', 'Monto Acumulado', 'Uso de los recursos', 'Plazo',
                    'Sector', 'Ingresos', 'Ubicación', 'Estrato Mínimo',
                    'Procesos judiciales', 'Alertas', 'Score Bureau Empresa', 'Huellas de Consulta', 'Tiempo en el negocio',
                    'Website empresa', 'Instagram empresa', 'LinkedIn empresa',
                    'LinkedIn empresarios', 'Edad empresarios', 'Activador', 'Número de accionistas',
                    'Impacto', 'Acceso previso a la banca', '# Empleados',
                    'Mujeres empresarias', 'Mujeres en cargos directivos']

        self.data = self.data.drop(columns=cat_cols)
        return self.data

    def merge_two_tables(self):
        """Merging two tables: Numerical and cathegorical"""
        self.data = self.data.merge(
            self.cat_data, left_on='client_id', right_on='ID Cliente')
        return self.data

    @staticmethod
    def red_state(data):
        count = 0
        cond = 1
        for i, value in enumerate(data):
            if value == 0:
                count += 1
            if i >= 4:
                if value >= 15:
                    cond = 0
                
        if count <= 2 and cond == 0 and sum(data) > 30:
            return 'LATE'
        elif sum(data) > 30:
            return 'UNDEF'
        else:
            return 'PAID'
    
    def customer_level(self):
        """Compress dataframe into a better study"""
        self.data = self.data.groupby(['client_id', 'loan_id']).agg({
            'state': lambda x: x.iloc[-1],  # Devuelve el último state
            'arrears_days': lambda x:  list(x),
            'total_paid': 'sum'}).round(0)  # Se suman todas las facturas
        
        self.data['state'] = self.data['arrears_days'].apply(Processing_data.red_state)
        self.data['arrears_days'] = self.data['arrears_days'].apply(lambda x: sum(x))
        self.data.reset_index(inplace=True)
        return self.data

    def head(self):
        """print head of df"""
        return self.data.head()

    # Funcion compacta de limpieza de data
    def data_cleaning(self):
        """This function resumes all the steps of organizing and cleaning the datasets"""
        self.data = self.fill_na_json()
        self.data = self.clean_data()
        self.data = self.delete_unnecessary_columns()
        self.data = self.organizing_data()
        self.cat_data = self.checking_cat_data()
        self.data = self.merge_two_tables()
        self.data = self.deleting_cat_info()
        self.data = self.customer_level()
        self.data = self.merge_two_tables()
        return self.data
        
# ---------------------------------------

    def transform_data(self):
        """transform_data : Transforma los datos numericos con el metodo PowerTransformer"""
        self.data = self.data_cleaning()
        self.data.drop(["client_id", "loan_id", "ID Cliente"], axis= 1, inplace=True)
        #if self.cleaning == True:
        #    self.data = self.data_cleaning()
        #else:
        #    self.cat_data["ID Cliente"] = self.data['client_id']
        #    self.data.drop(["ID Cliente"], axis= 1, inplace=True)
        #    self.data = self.merge_two_tables()
            
        for c in self.data.select_dtypes(exclude='object').columns:
            pt = PowerTransformer()
            self.data[c] = pt.fit_transform(
                np.array(self.data[c]).reshape(-1, 1))
        return self.data

    def reduction_dim(self):
        """reduction: Reduce la dimensionalidad de los datos aplicando Analisis Factorial de Datos Mixtos(FAMD)"""
        self.data = self.transform_data()
        
        self.data['state'] = self.data['state'].replace(to_replace="LATE",value="0")
        self.data['state'] = self.data['state'].replace(to_replace="PAID",value="1")
        self.data['state'] = self.data['state'].astype(object)

        # Declarar metodo para aplicar FAMD
        famd = prince.FAMD(
            n_components=2,
            n_iter=3,
            copy=True,
            check_input=True,
            engine='auto',
            random_state=42)

        # Ajustar y transformar la dimensión aplicando FAMD
        famd = famd.fit(self.data)
        transformada = famd.transform(self.data)

        Y = transformada.to_numpy()
        principalDf_famd = pd.DataFrame(
            data=Y, columns=['principal component 1', 'principal component 2'])
        finalDf_Cat_famd = pd.concat(
            [principalDf_famd, self.data['arrears_days']], axis=1, ignore_index=True)

        self.data = finalDf_Cat_famd
        return self.data

    def execute_model(self):
        """execute: Función que ejecuta el modelo con los datos procesados"""
        self.data = self.reduction_dim().to_numpy()
        self.kproto = KPrototypes(n_clusters=2, init = 'Huang', max_iter=100, verbose = 1,
                                  n_init = 15, random_state=4444, n_jobs=-1, gamma=.25)
        self.clusters = self.kproto.fit_predict(self.data, categorical=[2])
        #self.kproto = kproto
        #print(self.kproto)
        print(self.clusters)
        return self.clusters

    def guardar_model(self):
        """Saves the model into a joblib file"""
        try:
            joblib.dump(self.execute_model(), 'testeo.joblib')
            print("Saved correctly!")
        except:
            print("There is a problem to save the model, check on documentation")


class Testing_model(Processing_data):
    """This class test the model"""

    def __init__(self, data):
        """Testing model constructor"""
        #super(kproto, clusters).__init__()
        self.data = data
        self.kproto = []
        self.clusters = []
    
    def transform_data(self):
        """Transform data"""
        print("here")
        #self.data.replace([np.inf, -np.inf], np.nan, inplace=True)
        self.data.drop(["client_id",'loan_id', 'ID Cliente'], axis= 1, inplace=True)
        for c in self.data.select_dtypes(exclude='object').columns:
            pt = PowerTransformer()
            self.data[c] = pt.fit_transform(
                np.array(self.data[c]).reshape(-1, 1))
            print(self.data[c])
        #self.data.fillna(self.data.mean())
        #print(self.data.isnull().sum())
        return self.data


In [18]:
path_file = "../data/loan_schedules_2.xlsx"
cols = ['client_id',
        'loan_schedule_id',
        'loan_id',
        'index',
        'expected_date',
        'state',
        'arrears_days',
        'expected_detail',
        'paid_detail']
data = pd.read_excel(path_file, sheet_name = 0, header = 0, usecols = cols).replace('NaT', np.nan)

In [19]:
path_file = "../data/data_sempli_holberton.xlsx"
cat_cols = ['ID Cliente', 'Monto Acumulado', 'Uso de los recursos', 'Plazo',
       'Sector', 'Ingresos', 'Ubicación', 'Estrato Mínimo',
       'Procesos judiciales', 'Alertas', 'Score Bureau Empresa', 'Huellas de Consulta', 'Tiempo en el negocio',
       'Website empresa', 'Instagram empresa', 'LinkedIn empresa',
       'LinkedIn empresarios', 'Edad empresarios', 'Activador', 'Número de accionistas',
       'Impacto', 'Acceso previso a la banca', '# Empleados',
       'Mujeres empresarias', 'Mujeres en cargos directivos']
cat_data = pd.read_excel(path_file, sheet_name = 0, header = 2, usecols = cat_cols).replace('SIN INFO', np.nan)
cat_data = cat_data.dropna()

In [30]:
# Instanciar la clase para etapa de limpieza y ejcutar el modelo
new_data = Processing_data(data, cat_data)

In [34]:
new_data.transform_data()

overflow encountered in power
overflow encountered in multiply
overflow encountered in reduce


,state,arrears_days,total_paid,ID Cliente,Monto Acumulado,Uso de los recursos,Plazo,Sector,Ingresos,Ubicación,Estrato Mínimo,Procesos judiciales,Alertas,Score Bureau Empresa,Huellas de Consulta,Tiempo en el negocio,Website empresa,Instagram empresa,LinkedIn empresa,LinkedIn empresarios,Edad empresarios,Activador,Número de accionistas,Impacto,Acceso previso a la banca,# Empleados,Mujeres empresarias,Mujeres en cargos directivos
0,LATE,0.068201,1.990516,-2.323541,0.247658,CR,25 a 36 meses,Servicios,Seed,Medellín,0.481027,No,No,0.592443,0.514087,0.824309,Si,Si,Si,Si,0.542674,Internet,0.017484,No,Si,0.176367,No,0.447494
1,LATE,0.219112,1.308246,-2.323541,0.247658,CR,25 a 36 meses,Servicios,Seed,Medellín,0.481027,No,No,0.592443,0.514087,0.824309,Si,Si,Si,Si,0.542674,Internet,0.017484,No,Si,0.176367,No,0.447494
2,PAID,-0.027869,-0.515458,-2.033775,-1.405435,KT - EX,13 a 24 meses,Servicios,Seed,Medellín,-1.519614,No,No,0.585245,-0.688637,0.456239,Si,No,Si,No,-0.942694,SocialNetworks,0.736474,No,Si,-1.700640,No,-1.459134
3,LATE,0.735415,-0.901116,-1.901462,-1.626569,SP,13 a 24 meses,Industria,Seed,Medellín,-0.523889,No,No,0.145027,-1.130869,0.638175,Si,No,Si,Si,0.451404,CreditProfessionalContact,0.017484,No,Si,-1.107593,No,-1.459134
4,LATE,0.735415,-0.946665,-1.901462,-1.626569,SP,13 a 24 meses,Industria,Seed,Medellín,-0.523889,No,No,0.145027,-1.130869,0.638175,Si,No,Si,Si,0.451404,CreditProfessionalContact,0.017484,No,Si,-1.107593,No,-1.459134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,LATE,0.335600,-0.475159,1.951817,-1.405435,KT - CO,13 a 24 meses,Servicios,Seed,Bogotá D.C.,0.481027,No,No,0.900597,-0.688637,1.014737,Si,Si,No,Si,-0.054151,SocialNetworks,0.017484,No,Si,-0.896798,No,-1.459134
573,PAID,-0.613883,0.238679,2.032265,0.012235,KT - EX,25 a 36 meses,Servicios,Venture,Bogotá D.C.,-0.523889,No,Si,0.358389,0.069432,0.278411,Si,No,Si,Si,-0.166537,CommunicationMedia,-1.311581,No,Si,0.295523,No,0.867677
574,LATE,0.676864,-0.151688,2.040592,-0.436514,CR,25 a 36 meses,Servicios,Venture,Bogotá D.C.,-0.523889,No,No,0.492277,-0.688637,0.824309,No,No,No,No,-0.530803,Internet,0.736474,Si,Si,0.038432,No,1.411840
575,PAID,-1.494842,1.209378,2.084151,0.750264,KT - EX,25 a 36 meses,Servicios,Venture,Medellín,1.493650,No,Si,0.556522,-0.688637,-0.065288,Si,Si,No,Si,-0.166537,SocialNetworks,0.017484,Si,Si,-0.124416,No,2.024831


In [39]:
excel_final = pd.read_csv('../data/data_test_v5.csv')
excel_final['state'].value_counts()

PAID     344
LATE     164
UNDEF     69
Name: state, dtype: int64

In [36]:
excel_final.head()

,Unnamed: 0,client_id,loan_id,state,arrears_days,total_paid,ID Cliente,Monto Acumulado,Uso de los recursos,Plazo,Sector,Ingresos,Ubicación,Estrato Mínimo,Procesos judiciales,Alertas,Score Bureau Empresa,Huellas de Consulta,Tiempo en el negocio,Website empresa,Instagram empresa,LinkedIn empresa,LinkedIn empresarios,Edad empresarios,Activador,Número de accionistas,Impacto,Acceso previso a la banca,# Empleados,Mujeres empresarias,Mujeres en cargos directivos
0,0,241,1076,PAID,17,147915805.0,241.0,250000000,CR,25 a 36 meses,Servicios,Seed,Medellín,5,No,No,925.0,8.0,2016,Si,Si,Si,Si,45,Internet,2.0,No,Si,16,No,2
1,1,241,1244,PAID,23,91852726.0,241.0,250000000,CR,25 a 36 meses,Servicios,Seed,Medellín,5,No,No,925.0,8.0,2016,Si,Si,Si,Si,45,Internet,2.0,No,Si,16,No,2
2,2,362,1469,PAID,14,15211138.0,362.0,50000000,KT - EX,13 a 24 meses,Servicios,Seed,Medellín,3,No,No,924.0,2.0,2014,Si,No,Si,No,32,SocialNetworks,3.0,No,Si,3,No,0
3,3,428,119,UNDEF,64,8713879.0,428.0,40000000,SP,13 a 24 meses,Industria,Seed,Medellín,4,No,No,861.0,1.0,2015,Si,No,Si,Si,44,CreditProfessionalContact,2.0,No,Si,5,No,0
4,4,428,120,UNDEF,64,8102608.0,428.0,40000000,SP,13 a 24 meses,Industria,Seed,Medellín,4,No,No,861.0,1.0,2015,Si,No,Si,Si,44,CreditProfessionalContact,2.0,No,Si,5,No,0


In [31]:
%%time
cluster = new_data.execute_model()
cluster

overflow encountered in power
overflow encountered in multiply
overflow encountered in reduce


Best run was number 14
[1 1 0 0 0 0 0 1 1 1 1 0 1 1 1 0 0 0 1 1 1 1 1 0 1 0 0 0 0 1 1 0 0 1 1 1 1
 1 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 0 1 1 1
 0 0 1 0 0 1 1 0 0 1 1 1 0 1 1 1 0 0 1 1 0 1 0 1 0 0 0 1 1 1 1 1 1 1 0 0 0
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1
 1 0 0 0 0 0 0 1 0 0 1 1 1 1 1 1 0 0 0 0 0 0 1 1 0 1 1 1 1 1 0 0 1 1 0 0 0
 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 0 0 1 1 1 1 1 1
 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 1 1 1 0 0 0 0 1 1 0 0 0 1 0 0 1 1 0 0 0
 0 0 1 1 1 1 0 1 1 1 1 0 1 1 1 0 1 1 0 0 0 0 0 1 1 1 0 0 0 0 1 1 0 0 0 0 1
 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 0 0 0 1 0 1 1 1 0 0
 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1 1 1 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 1 0 0 1 0 1 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 1 1 1 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0
 0 0 0 1 1 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 1 1 0 0 0
 0

array([1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,

In [38]:
cluster

array([1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,

In [40]:
excel_final['cluster'] = cluster
#clusters_kproto_pca.reset_index(inplace=True)
excel_final.drop(['Unnamed: 0', 'ID Cliente'],axis=1, inplace=True)
excel_final.head()

,client_id,loan_id,state,arrears_days,total_paid,Monto Acumulado,Uso de los recursos,Plazo,Sector,Ingresos,Ubicación,Estrato Mínimo,Procesos judiciales,Alertas,Score Bureau Empresa,Huellas de Consulta,Tiempo en el negocio,Website empresa,Instagram empresa,LinkedIn empresa,LinkedIn empresarios,Edad empresarios,Activador,Número de accionistas,Impacto,Acceso previso a la banca,# Empleados,Mujeres empresarias,Mujeres en cargos directivos,cluster
0,241,1076,PAID,17,147915805.0,250000000,CR,25 a 36 meses,Servicios,Seed,Medellín,5,No,No,925.0,8.0,2016,Si,Si,Si,Si,45,Internet,2.0,No,Si,16,No,2,1
1,241,1244,PAID,23,91852726.0,250000000,CR,25 a 36 meses,Servicios,Seed,Medellín,5,No,No,925.0,8.0,2016,Si,Si,Si,Si,45,Internet,2.0,No,Si,16,No,2,1
2,362,1469,PAID,14,15211138.0,50000000,KT - EX,13 a 24 meses,Servicios,Seed,Medellín,3,No,No,924.0,2.0,2014,Si,No,Si,No,32,SocialNetworks,3.0,No,Si,3,No,0,0
3,428,119,UNDEF,64,8713879.0,40000000,SP,13 a 24 meses,Industria,Seed,Medellín,4,No,No,861.0,1.0,2015,Si,No,Si,Si,44,CreditProfessionalContact,2.0,No,Si,5,No,0,0
4,428,120,UNDEF,64,8102608.0,40000000,SP,13 a 24 meses,Industria,Seed,Medellín,4,No,No,861.0,1.0,2015,Si,No,Si,Si,44,CreditProfessionalContact,2.0,No,Si,5,No,0,0


In [43]:
excel_final['cluster'].value_counts()

0    310
1    267
Name: cluster, dtype: int64

In [44]:
excel_final.to_csv('../data/clusterizacion_v5.csv',encoding='utf-8-sig')

In [77]:
%%time
new_data.guardar_model()

overflow encountered in power
overflow encountered in multiply
overflow encountered in reduce


Best run was number 3
KPrototypes(gamma=0.25, n_clusters=2, n_init=15, n_jobs=-1, random_state=4444,
            verbose=1)
Se ha guardado correctamente!
Wall time: 17 s
